In [1]:
#importing libraries
import os
import sys
import requests


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.metrics import plot_roc_curve
#from sklearn.model_selection import accuracy_score,classification_report


In [2]:
price=pd.read_csv('price_data.csv')
price


,close,datetime,high,low,open,symbol,volume
0,105.51,1513144800000,107.11,105.4800,106.70,JPM,15020293
1,104.66,1513231200000,106.45,104.6400,106.12,JPM,13034232
2,106.14,1513317600000,106.52,105.2000,105.79,JPM,29350598
3,106.96,1513576800000,107.63,106.4800,107.19,JPM,12713148
4,106.51,1513663200000,107.49,106.4400,107.35,JPM,12149937
...,...,...,...,...,...,...,...
2515,67.77,1575871200000,69.50,66.5918,67.27,ARWR,2088509
2516,70.86,1575957600000,72.17,68.0000,68.02,ARWR,2281095
2517,68.22,1576044000000,71.80,67.2800,70.62,ARWR,1718444
2518,67.32,1576130400000,69.95,65.8900,68.28,ARWR,2050860


In [3]:
#rearrange rows
clean_price=price.drop(['datetime'],axis=1)
clean_price

,close,high,low,open,symbol,volume
0,105.51,107.11,105.4800,106.70,JPM,15020293
1,104.66,106.45,104.6400,106.12,JPM,13034232
2,106.14,106.52,105.2000,105.79,JPM,29350598
3,106.96,107.63,106.4800,107.19,JPM,12713148
4,106.51,107.49,106.4400,107.35,JPM,12149937
...,...,...,...,...,...,...
2515,67.77,69.50,66.5918,67.27,ARWR,2088509
2516,70.86,72.17,68.0000,68.02,ARWR,2281095
2517,68.22,71.80,67.2800,70.62,ARWR,1718444
2518,67.32,69.95,65.8900,68.28,ARWR,2050860


In [9]:
clean_price=clean_price[['symbol','high','low','open','close','volume']]
clean_price

,symbol,high,low,open,close,volume
0,JPM,107.11,105.4800,106.70,105.51,15020293
1,JPM,106.45,104.6400,106.12,104.66,13034232
2,JPM,106.52,105.2000,105.79,106.14,29350598
3,JPM,107.63,106.4800,107.19,106.96,12713148
4,JPM,107.49,106.4400,107.35,106.51,12149937
...,...,...,...,...,...,...
2515,ARWR,69.50,66.5918,67.27,67.77,2088509
2516,ARWR,72.17,68.0000,68.02,70.86,2281095
2517,ARWR,71.80,67.2800,70.62,68.22,1718444
2518,ARWR,69.95,65.8900,68.28,67.32,2050860


In [10]:
#finding the change in price
clean_price['change_in_price']=clean_price['close'].diff()

clean_price

,symbol,high,low,open,close,volume,change_in_price
0,JPM,107.11,105.4800,106.70,105.51,15020293,NaN
1,JPM,106.45,104.6400,106.12,104.66,13034232,-0.85
2,JPM,106.52,105.2000,105.79,106.14,29350598,1.48
3,JPM,107.63,106.4800,107.19,106.96,12713148,0.82
4,JPM,107.49,106.4400,107.35,106.51,12149937,-0.45
...,...,...,...,...,...,...,...
2515,ARWR,69.50,66.5918,67.27,67.77,2088509,0.50
2516,ARWR,72.17,68.0000,68.02,70.86,2281095,3.09
2517,ARWR,71.80,67.2800,70.62,68.22,1718444,-2.64
2518,ARWR,69.95,65.8900,68.28,67.32,2050860,-0.90


In [11]:
#finding where the rows 
mask=clean_price['symbol'] != clean_price['symbol'].shift(1)

#change the change in price at these points to be equal to null
clean_price['change_in_price']=np.where(mask==True,np.nan,clean_price['change_in_price'])

clean_price[clean_price.isna().any(axis=1)]

,symbol,high,low,open,close,volume,change_in_price
0,JPM,107.11,105.48,106.70,105.51,15020293,NaN
504,COST,190.12,187.80,188.53,188.28,3062856,NaN
1008,IBM,156.73,153.89,156.60,153.91,5661618,NaN
1512,HD,183.67,182.00,182.01,183.03,5177363,NaN
2016,ARWR,3.41,3.01,3.30,3.40,2037395,NaN


In [7]:
clean_price.shape()

TypeError: 'tuple' object is not callable

# Using Indicators
We use a couple of indicators to show the momentum direction. We will use Relative Strength Index, Exponential Stochastic Oscillator

In [24]:
n=14
up_df, down_df = clean_price[['symbol','change_in_price']].copy(), clean_price[['symbol','change_in_price']].copy()
down_df

,symbol,change_in_price
0,JPM,NaN
1,JPM,-0.85
2,JPM,1.48
3,JPM,0.82
4,JPM,-0.45
...,...,...
2515,ARWR,0.50
2516,ARWR,3.09
2517,ARWR,-2.64
2518,ARWR,-0.90


In [25]:
down_df

,symbol,change_in_price
0,JPM,NaN
1,JPM,-0.85
2,JPM,1.48
3,JPM,0.82
4,JPM,-0.45
...,...,...
2515,ARWR,0.50
2516,ARWR,3.09
2517,ARWR,-2.64
2518,ARWR,-0.90


In [13]:
#changing the negative values to 0 since the indicator doesn't allow it
up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0
up_df

,symbol,change_in_price
0,JPM,NaN
1,JPM,0.00
2,JPM,1.48
3,JPM,0.82
4,JPM,0.00
...,...,...
2516,ARWR,3.09
2517,ARWR,0.00
2518,ARWR,0.00
2519,ARWR,0.00


# Calculating RSI
We use the RSI formula

Exponential Weighted Moving Average (EWMA) is a statistical method used to calculate the weighted 
average of a series of data points, with more recent data points receiving higher weights. 

In [26]:

n=14
#for updays if the change is less than zero, set to o
up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0
#if the change for downdays is greater than 0, set to zero
down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

#making the down_df change in price to be absolute
down_df['change_in_price']=down_df['change_in_price'].abs()

#Using exponential weighted moving average.
ewma_up = up_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
ewma_up


0                       NaN
1                  0.000000
2                  0.792857
3                  0.803226
4                  0.557496
                     ...   
2516               2.056687
2517               1.782462
2518               1.544800
2519               1.338827
change_in_price    0.000000
Name: change_in_price, Length: 2521, dtype: float64

In [27]:
down_df

,symbol,change_in_price
0,JPM,NaN
1,JPM,0.85
2,JPM,0.00
3,JPM,0.00
4,JPM,0.45
...,...,...
2516,ARWR,0.00
2517,ARWR,2.64
2518,ARWR,0.90
2519,ARWR,0.86


In [28]:
ewma_down=down_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
ewma_down

0                       NaN
1                  0.850000
2                  0.394643
3                  0.243888
4                  0.306943
                     ...   
2516               0.866995
2517               1.103395
2518               1.076276
2519               1.047439
change_in_price    0.000000
Name: change_in_price, Length: 2521, dtype: float64

In [29]:
#Calculate the relative strength
relative_strength = ewma_up / ewma_down

#calculating the relative strength index using the formula
relative_strength_index= 100.0-(100.0/(1.0+relative_strength))
relative_strength_index

0                        NaN
1                   0.000000
2                  66.766917
3                  76.708553
4                  64.492214
                     ...    
2516               70.345789
2517               61.765421
2518               58.937633
2519               56.105517
change_in_price          NaN
Name: change_in_price, Length: 2521, dtype: float64

In [30]:
#Add the info to the data
clean_price['down_days']=down_df['change_in_price']
clean_price['up_days']=up_df['change_in_price']
clean_price['RSI']=relative_strength_index

clean_price.head(30)

,symbol,high,low,open,close,volume,change_in_price,down_days,up_days,RSI
0,JPM,107.1100,105.4800,106.70,105.51,15020293,NaN,NaN,NaN,NaN
1,JPM,106.4500,104.6400,106.12,104.66,13034232,-0.85,0.85,0.00,0.000000
2,JPM,106.5200,105.2000,105.79,106.14,29350598,1.48,0.00,1.48,66.766917
3,JPM,107.6300,106.4800,107.19,106.96,12713148,0.82,0.00,0.82,76.708553
4,JPM,107.4900,106.4400,107.35,106.51,12149937,-0.45,0.45,0.00,64.492214
5,JPM,107.7305,106.0900,107.60,106.14,13091473,-0.37,0.37,0.00,56.027105
6,JPM,108.0700,106.5600,106.59,107.83,12323670,1.69,0.00,1.69,74.007671
7,JPM,108.4600,106.8050,108.20,107.45,14573385,-0.38,0.38,0.00,66.909435
8,JPM,107.9900,106.4000,107.20,107.02,7403739,-0.43,0.43,0.00,59.462925
9,JPM,107.2900,106.6700,106.67,107.22,9496548,0.20,0.00,0.20,61.747645


# The Stochastic Oscillator
We shall use the next indicator, Stochastic Oscillator. Stochastic Oscillator follows the speed or the momentum of the price. As a rule, momentum changes before the price changes. 
It measures the level of the closing price relative to the low-high range over a period of time.

Formula is given below


In [43]:
low_14, high_14=clean_price[['symbol', 'low']].copy(), clean_price[['symbol', 'high']].copy()
low_14

,symbol,low
0,JPM,105.4800
1,JPM,104.6400
2,JPM,105.2000
3,JPM,106.4800
4,JPM,106.4400
...,...,...
2515,ARWR,66.5918
2516,ARWR,68.0000
2517,ARWR,67.2800
2518,ARWR,65.8900


In [44]:
#Group by symbol, then apply the rolling function and get the min and max
#low_14=low_14.groupby('symbol')['low'].transform(lambda x:x.rolling(window=n).min())
high_14=high_14.groupby('symbol')['high'].transform(lambda x:x.rolling(window=n).max())

low_14 = low_14.groupby('symbol')['low'].transform(lambda x: x.rolling(window = n).min())
high_14

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
2515    73.72
2516    73.72
2517    73.72
2518    73.72
2519    73.72
Name: high, Length: 2520, dtype: float64

In [46]:
#writing the formula in code
k_percent=100*((clean_price['close']- low_14)/ (high_14-low_14))
#k_percent

In [48]:
#add the info to the data
clean_price['low_14']=low_14
clean_price['high_14']=high_14
clean_price['k_percent']=k_percent

clean_price.head(30)

,symbol,high,low,open,close,volume,change_in_price,down_days,up_days,RSI,low_14,high_14,k_percent
0,JPM,107.1100,105.4800,106.70,105.51,15020293,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JPM,106.4500,104.6400,106.12,104.66,13034232,-0.85,0.85,0.00,0.000000,NaN,NaN,NaN
2,JPM,106.5200,105.2000,105.79,106.14,29350598,1.48,0.00,1.48,66.766917,NaN,NaN,NaN
3,JPM,107.6300,106.4800,107.19,106.96,12713148,0.82,0.00,0.82,76.708553,NaN,NaN,NaN
4,JPM,107.4900,106.4400,107.35,106.51,12149937,-0.45,0.45,0.00,64.492214,NaN,NaN,NaN
5,JPM,107.7305,106.0900,107.60,106.14,13091473,-0.37,0.37,0.00,56.027105,NaN,NaN,NaN
6,JPM,108.0700,106.5600,106.59,107.83,12323670,1.69,0.00,1.69,74.007671,NaN,NaN,NaN
7,JPM,108.4600,106.8050,108.20,107.45,14573385,-0.38,0.38,0.00,66.909435,NaN,NaN,NaN
8,JPM,107.9900,106.4000,107.20,107.02,7403739,-0.43,0.43,0.00,59.462925,NaN,NaN,NaN
9,JPM,107.2900,106.6700,106.67,107.22,9496548,0.20,0.00,0.20,61.747645,NaN,NaN,NaN
